In [1]:
from os import listdir
from os.path import isfile, join
import sys
import copy
import re
import numpy as np

from scipy.signal import stft
import scipy.io as sio
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import csv

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from tqdm import tqdm
from utils import Person, simple_low_pass, exp_feat, window_spikes, analyze, do_tc_full
import pickle as pkl
import peakutils
import pywt

from jAER_utils.converter import aedat2numpy
from utils import analyze

import pandas as pd
import seaborn as sns
import cv2
from sklearn.feature_extraction.image import extract_patches_2d

In [ ]:
# general stuff
fs = 200  # sampling frequency of MYO
VERBOSE = True
data_dir = '/Users/enea/Dropbox/Capocaccia2019_Gesture_DVS_Myo/Dataset/'
classes = classes = ['pinky', 'elle', 'yo', 'index', 'thumb']
classes_dict = {'pinky': 0, 'elle': 1, 'yo': 2, 'index': 3, 'thumb': 4}
classes_inv = {v: k for k, v in classes_dict.items()}

Load all data into subject obects 

In [ ]:
subjects = {}
names = [name for name in listdir(data_dir) if "emg" in name]
for name in names:
    _emg = np.load(data_dir + '{}'.format(name)).astype('float32')
    _ann = np.concatenate([np.array(['none']), np.load(data_dir + '{}'.format(name.replace("emg","ann")))[:-1]])
    
    subjects[name.split("_")[1]] = Person(name.split("_")[1], _emg, _ann, classes=classes)
    
    if VERBOSE:
        print("Loaded {}: EMG = [{}] // ANN = [{}]".format(name.split("_")[1], _emg.shape, len(_ann)))
print("Data Loaded! {} Sessions".format(len(subjects.keys())))

In [ ]:
# separates data in correct trial type
for name, data in subjects.items():
    for _class in classes:
        _annotation = np.float32(data.ann == _class)
        derivative = np.diff(_annotation)/1.0
        begins = np.where(derivative == 1)[0]
        ends = np.where(derivative == -1)[0]
        for b, e in zip(begins, ends):
            _trials = data.emg[b:e]
            data.trials[_class].append(_trials)
            data.begs[_class].append(b)
            data.ends[_class].append(e)
print("Done sorting trials!")

Now every subject has different fields, each field is a dictionary and the keys are the gestures.
We have the following fields: begs (beginning time of trials), ends (end time of trials), trials (actual myo data)

In [ ]:
# example, see that each gesture is 2 seconds
print(np.array(subjects['Gemma1'].begs['elle']) / fs)
print(np.array(subjects['Gemma1'].ends['elle']) / fs)

In [ ]:
# this is to find the zero timestepping of the davis
def find_trigger(ts):
    return np.where(np.diff(ts) < 0)[0][0]

def create_frame(x, y, dim=(128, 128)):
    img = np.zeros(dim)
    for _x, _y in zip(x.astype('int32'), y.astype('int32')):
        img[dim[0] - 1 - _x,_y] += 1
    return np.log10(img / np.max(img) + 1e-10)

In [ ]:
subject = 'Bruno2'
# decoders takes some time since it is a very long recordings, especially with DAVIS
events = aedat2numpy(data_dir + subject +'_dvs.aedat')
events = events[:, find_trigger(events[2]):]
events[2] = events[2] / 1e3
print(events.shape)

In [ ]:
gesture = 'thumb'
trial = 3
# load beginning and end 
b = np.array(subjects[subject].begs[gesture][trial]) / fs
e = np.array(subjects[subject].ends[gesture][trial]) / fs

frame_size = 1.
shift = 1.
beginning = b + shift
ending = beginning + frame_size
# slice
sl = (events[2] > beginning) & (events[2] < ending)

img = events[:, sl]
print(img.shape)
fig, ax = plt.subplots(1, 2, figsize=(20,10))

beg_int = int(shift * fs)
end_int = int((shift + frame_size) * fs)

# dd = subjects[subject].trials[gesture][trial][beg_int:end_int]
# feat = analyze(dd, frame_len=0.01, frame_step=0.005, feat='RMS')
# for i in range(8):
#     _ = ax[1].plot(dd[:, i], label='channel{}'.format(i))
# ax[1].legend()

sll = (img[1] > 20) & (img[1] < 110) & (img[0] > 20) & (img[0] < 110)

frame = np.uint8(create_frame(img[1][sll], img[0][sll]) * 254)
blurred = cv2.GaussianBlur(frame, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

M = cv2.moments(thresh)
x = int(M["m10"] / M["m00"])
y = int(M["m01"] / M["m00"])

_ = ax[0].imshow(frame)
ax[0].plot([x - 30, x + 30, x + 30, x - 30, x - 30], 
           [y  - 30, y  - 30, y + 30, y  + 30, y  - 30], '--', linewidth=7)

_ = ax[1].imshow(frame[y - 30: y + 30, x-30:x+30])



In [ ]:
a = (img[0] > (x - 30)) & (img[0] < (x + 30))
b = (img[1] > (127 - y - 30)) & (img[1] < (127 - y + 30))
xx = img[0][a & b]
yy = img[1][a & b]
ts = img[2][a & b]
pol = img[3][a & b]

plt.plot(xx, yy, '.')

In [ ]:
for subject, data in subjects.items():

    # decoders takes some time since it is a very long recordings, especially with DAVIS
    events = aedat2numpy(data_dir + subject +'_dvs.aedat')
    events = events[:, find_trigger(events[2]):]
    events[2] = events[2] / 1e3
    
    for gesture in classes:
        for trial in range(5):
            print("{} :: {} :: {}".format(subject, gesture, trial))
            # load beginning and end 
            b = np.array(data.begs[gesture][trial]) / fs
            e = np.array(data.ends[gesture][trial]) / fs

            frame_size = 2
            shift = 0.
            beginning = b + shift
            ending = beginning + frame_size

            # slice
            sl = (events[2] > beginning) & (events[2] < ending)

            img = events[:, sl]

            beg_int = int(shift * fs)
            end_int = int((shift + frame_size) * fs)

            sll = (img[1] > 20) & (img[1] < 110) & (img[0] > 20) & (img[0] < 110)

            frame = np.uint8(create_frame(img[1][sll], img[0][sll]) * 254)
            blurred = cv2.GaussianBlur(frame, (5, 5), 0)
            thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

            M = cv2.moments(thresh)
            x = int(M["m10"] / M["m00"])
            y = int(M["m01"] / M["m00"])

            a = (img[0] > (x - 30)) & (img[0] < (x + 30))
            b = (img[1] > (127 - y - 30)) & (img[1] < (127 - y + 30))
            xx = img[0][a & b]
            yy = img[1][a & b]
            ts = img[2][a & b]
            pol = img[3][a & b]

            data.x[gesture].append(xx - min(xx))
            data.y[gesture].append(yy - min(yy))
            data.ts[gesture].append(ts - min(ts))
            data.pol[gesture].append(pol)


In [ ]:
pkl.dump(subjects, open('10_people_dvs_emg.pkl', 'wb'))

In [ ]:
data = pkl.load(open('10_people_dvs_emg.pkl', 'rb'))

In [ ]:
sub = 'Melika2'
trial = 1
gesture = 'index'
_x = data[sub].y[gesture][trial]
_y = data[sub].x[gesture][trial]
_ts = data[sub].ts[gesture][trial]
_p = data[sub].pol[gesture][trial]

fil = (_ts > 0.25) & (_ts < 1)
frame = create_frame(_x[fil], _y[fil], dim=(60, 60))
plt.imshow(frame)
plt.figure()
_ = plt.plot(data[sub].trials[gesture][trial])


frame = np.uint8(frame * 254)
blurred = cv2.GaussianBlur(frame, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

sift = cv2.xfeatures2d.SIFT_create()
(kps, descs) = sift.detectAndCompute(thresh, None)

print("# kps: {}, descriptors: {}".format(len(kps), descs.shape))

surf = cv2.xfeatures2d.SURF_create()
(kps, descs) = surf.detectAndCompute(thresh, None)
print("# kps: {}, descriptors: {}".format(len(kps), descs.shape))

In [ ]:
# extract sift and surf 
for subject, sub in data.items():
    for gesture in classes:
        for trial in range(5):

            _x = sub.y[gesture][trial]
            _y = sub.x[gesture][trial]
            _ts = sub.ts[gesture][trial]
            _p = sub.pol[gesture][trial]

            fil = (_ts > 0.200) & (_ts < 2)
            frame = create_frame(_x[fil], _y[fil], dim=(60, 60))

            frame = np.uint8(frame * 254)
            blurred = cv2.GaussianBlur(frame, (5, 5), 0)
            thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

            sift = cv2.xfeatures2d.SIFT_create()
            (kps, descs1) = sift.detectAndCompute(thresh, None)
            sub.sift[gesture].append(descs1)

            surf = cv2.xfeatures2d.SURF_create()
            (kps, descs2) = surf.detectAndCompute(thresh, None)
            sub.surf[gesture].append(descs2)
            
            print("{}::{}::{}::{}::{}".format(subject, gesture, trial, descs1.shape, descs2.shape))

In [ ]:
X1 = []
X2 = []
Y = []

for subject, sub in data.items():
    for gesture in classes:
        for trial in range(5):
            if sub.sift[gesture][trial].shape[0] >= 7 and sub.surf[gesture][trial].shape[0] >= 7: 
                X1.append(sub.sift[gesture][trial][:7])
                X2.append(sub.surf[gesture][trial][:7])
                Y.append(classes_dict[gesture])
            

X1 = np.array(X1)
X2 = np.array(X2)
X = np.hstack([X1.reshape(X1.shape[0], -1), X2.reshape(X1.shape[0], -1)])
Y = np.array(Y)

X -= np.mean(X, 0, keepdims=True)
X /= np.std(X, 0, keepdims=True) + 1e-15

print(np.array(X1).shape)
print(np.array(X2).shape)
print(np.array(Y).shape)


In [ ]:
print(do_tc_full(X, Y.reshape(-1,1 ), pca_comp=4, avg=True))

In [ ]:
# extract sift and surf 

X1 = []
X2 = []
Y = []

for subject, sub in data.items():
    if subject not in ['Enea3', 'Melika2']:
        for gesture in classes:
            for trial in range(5):

                _emg = sub.trials[gesture][trial]

                if _emg.shape[0] < 400:
                    _emg = np.vstack([_emg, np.zeros((400 - _emg.shape[0], 8))])
                _emg = _emg[80:400]
                X2.append(_emg.reshape(-1, 40, 8))

                _x = sub.y[gesture][trial]
                _y = sub.x[gesture][trial]
                _ts = sub.ts[gesture][trial]
                _p = sub.pol[gesture][trial]
                for i in [0.4, 0.6, 0.8, 1., 1.2, 1.4, 1.6, 1.8]:
                    fil = (_ts > i) & (_ts < i + .2)
                    frame = create_frame(_x[fil], _y[fil], dim=(60, 60))
                    X1.append(frame)
                    Y.append(classes_dict[gesture])
#                     print("{}::{}::{}::{}::{}".format(subject, gesture, trial, frame.shape, i))

In [ ]:
X1 = np.array(X1)
X2 = np.array(X2).reshape(-1, 40, 8)
Y = np.array(Y).reshape(-1, 1)

print(np.array(X1).shape)
print(np.array(X2).shape)
print(np.array(Y).shape)

# np.save('_cc19_hand_gestures_10p_img', X1)
np.save('_cc19_hand_gestures_10p_emg', X2)
np.save('_cc19_hand_gestures_10p_lbl', Y)

In [ ]:
print((X1[0] - np.min(X1[0]))

In [ ]:
fig, ax = plt.subplots(10, 10, figsize=(20, 20))

for i in range(10):
    for j in range(10):
        _idx = np.random.randint(5600)
        a = X1[_idx]
        a -= np.min(a)
        a /= np.max(a)
        frame = np.uint8(a * 255)
        blurred = cv2.GaussianBlur(frame, (5, 5), 0)
        thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
        ax[i][j].imshow(frame)
        ax[i][j].set_title(classes_inv[Y[_idx][0]])

In [ ]:
def extract_grid(img, size=(20, 20), shift=None):
    if shift == None:
        shift = size
    x, y = img.shape
    patches = []
    n_win_x = (x - size[0]) // shift[0] + 1
    n_win_y = (y - size[1]) // shift[1] + 1
    for i in range(n_win_x):
        for j in range(n_win_y):
            patches.append(img[i * shift[0]:i * shift[0] + size[0], j * shift[1]:j * shift[1] + size[1]])
            
    return patches

pp = extract_grid(X1[100], (16, 16), shift=(8, 8))
print(len(pp))
fig, ax = plt.subplots(6, 6)
for i, ppp in enumerate(pp):
    ax[i//6][i % 6].imshow(ppp, aspect='auto')

In [ ]:
# SIFT features codebook
all_V_sift = []
all_patches = []
sift = cv2.xfeatures2d.SIFT_create()

for j, img in enumerate(X1):
    a = img
    a -= np.min(a)
    a /= np.max(a)
    frame = np.uint8(a * 255)
    
    patches = extract_patches_2d(frame, (20, 20), max_patches=0.1)

    print("{}::{}/{}".format(len(patches), j, len(X1)))
    _patches = []
    for p in patches:
        
        (kps, descs1) = sift.detectAndCompute(p, None)
        if descs1 is not None:
            _patches.append(descs1)
    all_V_sift.append(_patches)
    
for_kmeans = np.vstack([np.vstack(i) for i in all_V_sift])
print(for_kmeans.shape)

In [ ]:
np.array(all_V_sift[0][0]).shape

In [ ]:
for_kmeans = np.vstack([np.vstack(i) for i in all_V_sift])
print(for_kmeans.shape)

In [ ]:
np.save('kepoints_20x20_168_sift', for_kmeans)

In [ ]:
for_kmeans = np.load('kepoints_20x20_168_sift.npy')
print(for_kmeans.shape)

In [ ]:
# kmeans
from sklearn.cluster import KMeans
k = 500
kmeans = KMeans(n_clusters=k, random_state=0).fit(for_kmeans[::5])
C_sift = kmeans.cluster_centers_
print(C_sift.shape)

In [ ]:
np.save('centers_20x20_840_sift', C_sift)

In [ ]:
# create histogram features
all_h_sift = []

for feat in enumerate(all_V_sift):
    
    h = np.zeros((k,))
    
    for p in feat:
        _id = np.argmin(np.linalg.norm(p - C_sift, axis=1))
        h[_id] += 1
                
    all_h_sift.append(h)

    
all_h_sift = np.array(all_h_sift)
print(all_h_sift.shape)


In [ ]:
# create histogram features
all_h_sift = []
sift = cv2.xfeatures2d.SIFT_create()

for j, img in enumerate(X1):
    a = img
    a -= np.min(a)
    a /= np.max(a)
    frame = np.uint8(a * 255)

    patches = extract_grid(frame)
    h = np.zeros((k,))
    
    for p in patches:
        (kps, descs1) = sift.detectAndCompute(p, None)
        
        if descs1 is not None:
            for d in descs1:
                _id = np.argmin(np.linalg.norm(d - C_sift, axis=1))
                h[_id] += 1
                
    all_h_sift.append(h)

    
all_h_sift = np.array(all_h_sift)
print(all_h_sift.shape)


In [ ]:
print(all_h_sift[0])

In [ ]:
_XX = np.array([i / np.linalg.norm(i) for i in all_h_sift])

print(do_tc_full(_XX, Y, avg=True, verbose=True))

In [ ]:
im = X1[2113]
im -= np.min(im)
im /= np.max(im)
print(im.mean())
fig, ax = plt.subplots(2, 3)
ax[0][0].imshow(im)

im = np.float32(im)
 
# Calculate gradient 
gx = cv2.Sobel(im, cv2.CV_32F, 1, 0, ksize=1)
gy = cv2.Sobel(im, cv2.CV_32F, 0, 1, ksize=1)
print(gy.shape)
ax[0][1].imshow(np.abs(gx))
ax[0][2].imshow(np.abs(gy))

# mag angle
mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
ax[1][0].imshow(np.abs(mag))
ax[1][1].imshow(angle)
plt.tight_layout()

patches = extract_grid(im, (8, 8))
kk = patches[17]
gx_p = cv2.Sobel(kk, cv2.CV_32F, 1, 0, ksize=1)
gy_p = cv2.Sobel(kk, cv2.CV_32F, 0, 1, ksize=1)

# mag angle
mag_p, angle_p = cv2.cartToPolar(gx_p, gy_p, angleInDegrees=True)
angle_p = np.abs(angle_p - 180)



In [ ]:
N_BUCKETS = 9
CELL_SIZE = 8  # Each cell is 8x8 pixels
BLOCK_SIZE = 2  # Each block is 2x2 cells

def assign_bucket_vals(m, d, bucket_vals):
    left_bin = int(d / 20.) % N_BUCKETS
    # Handle the case when the direction is between [160, 180)
    if (d == 180.):
        right_bin = 0
    else:
        right_bin = (int(d / 20.) + 1) % N_BUCKETS
    
    if left_bin < right_bin:
        left_val= m * (right_bin * 20 - d) / 20
        right_val = m * (d - left_bin * 20) / 20
    else:
        left_val= m * (d - left_bin * 20) / 20
        right_val = m * (right_bin * 20 - d) / 20 
    if left_val < 0:
        left_val = 0.
    if right_val < 0:
        right_val = 0.
    bucket_vals[left_bin] += np.abs(left_val)
    bucket_vals[right_bin] += np.abs(right_val)
    
all_veat = []
for j, im in enumerate(X1):
    im -= np.min(im)
    im /= np.max(im)
    im = np.float32(im)
    blocks = extract_grid(im, (16, 16), (8, 8))

    all_buckets = []
    for block in blocks:
        cells = extract_grid(block, (8, 8))
        to_concat = []
        for cell in cells:
            bucket_vals = np.zeros((N_BUCKETS))
            gx_p = cv2.Sobel(cell, cv2.CV_32F, 1, 0, ksize=1)
            gy_p = cv2.Sobel(cell, cv2.CV_32F, 0, 1, ksize=1)

            # mag angle
            mag_p, angle_p = cv2.cartToPolar(gx_p, gy_p, angleInDegrees=True)
            mag_p = np.abs(mag_p)
            angle_p = np.abs(angle_p - 180)
            for m, d in zip(mag_p.reshape(-1,), angle_p.reshape(-1)):
                assign_bucket_vals(m, d, bucket_vals)
            to_concat.append(bucket_vals)
        veat = np.array(to_concat)
        all_buckets.append(veat)

    veat = np.array(all_buckets)
    all_veat.append(veat)
    print(j)

print(np.array(all_veat).shape)

In [ ]:
eps = 1e-15
print(np.array(all_veat).shape)
all_veat_res = np.array(all_veat).reshape(5600, 36, -1)
# eps = 1e-7
all_veat_res /= np.sqrt(np.sum(all_veat_res ** 2, 2, keepdims=True) + eps + 1)
# all_veat_res = np.sqrt(all_veat_res)
# all_veat_res /= np.linalg.norm(all_veat_res, axis=2, keepdims=True) + eps
# all_veat_norm = all_veat_res / (np.linalg.norm(all_veat_res, axis=1, keepdims=True) + 1e-15)
# all_veat_norm = all_veat_res
all_veat_final = all_veat_res.reshape(5600, -1)
# all_veat_final = all_veat_final / (np.linalg.norm(all_veat_final, axis=1, keepdims=True) + 1e-15)

print(all_veat_final.shape)

In [ ]:
np.sum(all_veat_res ** 2, 2, keepdims=True).shape

In [ ]:
np.save('hog_16x16_8x8_sqrt_norm', all_veat_final)

In [ ]:
acc_tr, acc_te, _, _ = do_tc_full(all_veat_final, np.array(Y).reshape(-1,1), pca_comp=300, verbose=True, avg=True)

In [4]:
# emg = np.load('_cc19_hand_gestures_10p_emg.npy')
# emg_mav = np.array([analyze(i, frame_len=0.2, frame_step=0.2, feat='MAV') for i in emg])
# emg_sd = np.array([analyze(i, frame_len=0.2, frame_step=0.2, feat='SD') for i in emg])
# emg_rms = np.array([analyze(i, frame_len=0.2, frame_step=0.2, feat='RMS') for i in emg])

print(np.hstack([emg_mav, emg_rms, emg_sd]).shape)
emg = np.hstack([emg_mav, emg_rms, emg_sd]).reshape(-1, 24)
np.save('emg_mav_rms_sd_02', emg)

(5600, 3, 8)


In [5]:
dvs = np.load('hog_16x16_8x8_sqrt_norm.npy')
emg = np.load('emg_mav_rms_sd_02.npy')
lbl = np.load('_cc19_hand_gestures_10p_lbl.npy')
print(emg.shape)
print(dvs.shape)
print(lbl.shape)



(5600, 24)
(5600, 1296)
(5600, 1)


In [22]:
emg -= np.mean(emg, 0, keepdims=True)
emg /= np.std(emg, 0, keepdims=True) + 1e-15
acc_tr, acc_te, _, _ = do_tc_full(emg, lbl, pca_comp=0, verbose=False, avg=True, kernel='rbf')
print("{:.4} :: {:.4}".format(acc_tr, acc_te))

0.7881 :: 0.7666


In [23]:
dvs -= np.mean(dvs, 0, keepdims=True)
dvs /= np.std(dvs, 0, keepdims=True) + 1e-15
acc_tr, acc_te, _, _ = do_tc_full(dvs, lbl, pca_comp=0, verbose=True, avg=True, kernel='rbf')
print("{:.4} :: {:.4}".format(acc_tr, acc_te))

Train on: 5040 / Test on 560
Fold 0: Train acc: 0.957 || Test acc: 0.839
Fold 1: Train acc: 0.96 || Test acc: 0.82
Fold 2: Train acc: 0.959 || Test acc: 0.848
Fold 3: Train acc: 0.958 || Test acc: 0.834
Fold 4: Train acc: 0.958 || Test acc: 0.848
Fold 5: Train acc: 0.958 || Test acc: 0.839
Fold 6: Train acc: 0.961 || Test acc: 0.839
Fold 7: Train acc: 0.958 || Test acc: 0.843
Fold 8: Train acc: 0.959 || Test acc: 0.827
Fold 9: Train acc: 0.96 || Test acc: 0.852
0.9588 :: 0.8389


In [24]:
acc_tr, acc_te, _, _ = do_tc_full(np.hstack([emg, dvs]), lbl, pca_comp=0, verbose=True, avg=True, kernel='rbf')
print("{:.4} :: {:.4}".format(acc_tr, acc_te))

Train on: 5040 / Test on 560
Fold 0: Train acc: 0.981 || Test acc: 0.889
Fold 1: Train acc: 0.98 || Test acc: 0.896
Fold 2: Train acc: 0.979 || Test acc: 0.907
Fold 3: Train acc: 0.981 || Test acc: 0.909
Fold 4: Train acc: 0.978 || Test acc: 0.889
Fold 5: Train acc: 0.979 || Test acc: 0.884
Fold 6: Train acc: 0.981 || Test acc: 0.875
Fold 7: Train acc: 0.98 || Test acc: 0.855
Fold 8: Train acc: 0.98 || Test acc: 0.857
Fold 9: Train acc: 0.98 || Test acc: 0.868
0.9798 :: 0.883


In [ ]:
# Random location [200, 200] as an example.
loc_x = loc_y = 200

ydata = get_magnitude_hist_block(loc_x, loc_y)
ydata = ydata / np.linalg.norm(ydata)

xdata = range(len(ydata))
bucket_names = np.tile(np.arange(N_BUCKETS), BLOCK_SIZE * BLOCK_SIZE)

assert len(ydata) == N_BUCKETS * (BLOCK_SIZE * BLOCK_SIZE)
assert len(bucket_names) == len(ydata)

plt.figure(figsize=(10, 3))
plt.bar(xdata, ydata, align='center', alpha=0.8, width=0.9)
plt.xticks(xdata, bucket_names * 20, rotation=90)
plt.xlabel('Direction buckets')
plt.ylabel('Magnitude')
plt.grid(ls='--', color='k', alpha=0.1)
plt.title("HOG of block at [%d, %d]" % (loc_x, loc_y))
plt.tight_layout()

In [ ]:
all_hog = []

for img in X1:
    # HOG
    a = img
    a -= np.min(a)
    a /= np.max(a)
    frame = np.uint8(a * 255)

    patches = extract_grid(frame, (8, 8))

    all_hist = []

    for p in patches:
        gx = cv2.Sobel(p, cv2.CV_32F, 1, 0)
        gy = cv2.Sobel(p, cv2.CV_32F, 0, 1)


        mag, ang = cv2.cartToPolar(gx, gy)
        bin_n = 16
        bin = np.int32(bin_n*ang/(2*np.pi))
        bin_cells = bin[:100,:100], bin[100:,:100], bin[:100,100:], bin[100:,100:]
        mag_cells = mag[:100,:100], mag[100:,:100], mag[:100,100:], mag[100:,100:]
        hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
        hist = np.hstack(hists)
        all_hist.append(hist)
    # transform to Hellinger kernel
    hist = sum(all_hist)
    eps = 1e-7
    hist /= hist.sum() + eps
    hist = np.sqrt(hist)
    hist /= np.linalg.norm(hist) + eps

    all_hog.append(hist)
print(np.array(all_hog).shape)

In [ ]:
acc_tr, acc_te = do_tc_full(np.array(all_hog), np.array(Y).reshape(-1,1), verbose=True, avg=True)

In [ ]:
all_X, all_Y = [], []
n_ch = 8
preprocess = True
frame_len = 0.3
overlap = 0.0
frame_step = frame_len * (1. - overlap)
features = ['MAV', 'RMS', 'SD']
n_comp = 8
for i, idx in enumerate(X2):
    _sub = np.mean(np.abs(idx), 0)
    all_X.append(_sub)
    all_Y.append(Y[i])

all_X = np.vstack(all_X)
all_Y = np.vstack(all_Y)

all_X -= np.mean(all_X, 0, keepdims=True)
all_X /= np.std(all_X, 0, keepdims=True) + 1e-15

acc_tr, acc_te = do_tc_full(all_X, all_Y, verbose=True, pca_comp=n_comp, avg=False)
print("Train: {:.4} +/- {:.4}".format(np.mean(acc_tr), np.std(acc_tr)))
print("Test: {:.4} +/- {:.4}".format(np.mean(acc_te), np.std(acc_te)))

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
(kps, descs) = sift.detectAndCompute(frame, None)

print(descs.shape)

In [ ]:
import csv
with open('gesture.csv', 'w') as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerow(['x', 'y', 'type'])
    for i in range(img.shape[1]):
        csv_writer.writerow([img[0, i], img[1, i], img[3, i]])


In [ ]:
data = pd.read_csv('gesture.csv')

g = sns.jointplot("x", "y", data=data, size=20)
plt.savefig('spatial_density_yo.pdf')

In [ ]:
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
gm = GaussianMixture(4) 
gm.fit(img[0].reshape(-1, 1) / 180)

x = np.linspace(0, 1, 1000)
plt.hist(img[0] / 180, 50, density=True)
plt.plot(x.reshape(-1, 1), stats.norm.pdf(x, gm.means_[0], gm.covariances_[0]).T)
plt.plot(x.reshape(-1, 1), stats.norm.pdf(x, gm.means_[1], gm.covariances_[1]).T)
plt.plot(x.reshape(-1, 1), stats.norm.pdf(x, gm.means_[2], gm.covariances_[2]).T)

